In [1]:
import numpy as np
import MDAnalysis as mda
import sys
from MDAnalysis.analysis.distances import distance_array
import networkx as nx
import pandas as pd
import mdtraj as md
import matplotlib
import matplotlib.pyplot as plt
matplotlib.use("TkAgg")
import itertools
import time

In [2]:
# Queste sono da importare direttamente dal pdb ma ce da risolvere la numerazione

amino_dict_num = {0:'TYR', 1:'THR', 2:'ILE', 3:'ALA', 4:'ALA', 5:'LEU', 6:'LEU', 7:'SER', 8:'PRO', 9:'TYR', 10:'SER'}

pdb_resids = list(range(0, 11))

In [3]:
def contact_extraction(contact_list, contacts_histogram, pdb_resids):
    # Function to extract the informations from the pairs list
    for an in pdb_resids:
        # pdb_resids is the aminocid number
        # Selection of the rows contaning the aminoacid number of interest.
        # Note the | and not the &
        is_an = (contact_list['ai_resnumber'] == an) | (contact_list['aj_resnumber'] == an)

        # Filtering a dataframe containing only the aminoacid selected.
        contacts_an = contact_list[is_an]
        #print(pairs_an.to_string())

        print(f'\nTotal amount of contacts made by {amino_dict_num.get(an)}:', len(contacts_an))
        contacts_an.loc[contacts_an['ai_resnumber'] == an, 'an_with_who'] = contacts_an['aj_resname'] + '_' + contacts_an['aj_resnumber'].astype(str)
        contacts_an.loc[contacts_an['ai_resnumber'] != an, 'an_with_who'] = contacts_an['ai_resname'] + '_' + contacts_an['ai_resnumber'].astype(str)
        #print(pairs_an.to_string())
        
        # Create Pandas Series with contact counts for every aminoacid in an selection:
        # an_with_who count of the unique values in column "an_with_who" in pairs_an dataframe
        an_with_who = contacts_an.an_with_who.value_counts()
        print(f'\nList of contacts based on aminoacid:\n{an_with_who.to_string()}')

        # Change "an_with_who" in a dictionary to put in a matrix
        contacts_histogram['{0}'.format(an)] = an_with_who.to_dict()
        #print(f'\nAnd the consequent nested dictionary:\n{contacts_histogram}\n')
    
    return contacts_histogram#, sigma_histogram




def make_contacts_histo_df(contacts_histogram):
    # Contact dataframe preparation
    histo_df = pd.DataFrame(contacts_histogram)
    histo_df[''] = histo_df.index.astype(str)
    histo_df[''] = histo_df[''].str.split('_').str[1].astype(int)
    histo_df.sort_values(by = [''], inplace = True)
    histo_df.drop(columns='', inplace=True)
    histo_df = histo_df.fillna(0)
    histo_df.columns = list(amino_dict_num.values())
    print(histo_df.to_string())
    return histo_df

In [4]:
#ref_structure = fibril.pdb
#traj = start_clust-dt10000.xtc
Min_Distance = 4

In [5]:
#structure parameters

#topology = md.load(ref_structure).topology
topology = md.load('fibril.pdb').topology
#trajectory = md.load(traj, top=ref_structure)
trajectory = md.load('start_clust-dt10000.xtc', top='fibril.pdb')
frames=trajectory.n_frames				#Number of frames da provare a vedere cosa viene fuori con MDAnalysis
chains=topology.n_chains				#Number of chains

print(f'Number of chains', chains)

atoms=int(topology.n_atoms/chains)			#Number of atoms in each monomer 
AminoAcids = int(topology.n_residues/chains)#-2		#Number of residues per chain ('-2' avoid the N- and C- cap residues as individual residues)

isum=1
atoms_list=[]
atomsperAminoAcid=[]
residue_list=[]

for residue in topology.chain(0).residues:
    print(f'Residues in topology', residue)
    atoms_list.append(residue.n_atoms)
    residue_list.append(residue)
    ', '.join(map(lambda x: "'" + x + "'", str(residue_list)))
#The N- and C- cap residues are part of the 1st and last residue index. If no N- and C- cap residues for the protein, comment the line below using "#"
#del residue_list[0]; del residue_list[-1] 

for ii in range(len(atoms_list)):
    isum = isum + atoms_list[ii]
    atomsperAminoAcid.append(isum)
atomsperAminoAcid.insert(0, 1)

print('atomsperAminoAcid', atomsperAminoAcid) # Questo e' perche' le chains partono da 0 e i gruppi vengono ottenuti moltiplicando per il resid



#The N- and C- cap residues are part of the 1st and last residue index. If no N- and C- cap residues for the protein, comment the line below using "#"
#del atomsperAminoAcid[1]; del atomsperAminoAcid[-2]      

# Create Universe
print('Create Universe')
uni = mda.Universe('fibril.pdb','start_clust-dt10000.xtc')
n,t = list(enumerate(uni.trajectory))[0]
box = t.dimensions[:6]
print('Box dimensions', box)


atom_Groups = [[] for x in range(chains)]
m_start=0
for m in range(0,chains):
    # With this functions every chain is defined as a group of atoms
    # m is the chain number, atom is the number of atoms in every chain
    m_end = atoms * (m+1) # potevano evitare di fare questa cacata
    atom_Groups[m].extend([uni.select_atoms('bynum '+ str(m_start) + ':' + str(m_end))])
    m_start = m_end + 1



AtomGroups = [[] for x in range(chains)]
for m in range(0,chains):
    for aa in range(0,AminoAcids):
        #print(uni.select_atoms('bynum '+str(atoms*m + atomsperAminoAcid[aa])+':'+str(atoms*m + atomsperAminoAcid[aa + 1] - 1 )))
        AtomGroups[m].extend([uni.select_atoms('bynum '+str(atoms*m + atomsperAminoAcid[aa])+':'+str(atoms*m + atomsperAminoAcid[aa + 1] - 1 ))])


Number of chains 584
Residues in topology TYR1
Residues in topology THR2
Residues in topology ILE3
Residues in topology ALA4
Residues in topology ALA5
Residues in topology LEU6
Residues in topology LEU7
Residues in topology SER8
Residues in topology PRO9
Residues in topology TYR10
Residues in topology SER11
atomsperAminoAcid [1, 13, 20, 28, 33, 38, 46, 54, 60, 67, 79, 86]
Create Universe
Box dimensions [700. 700. 700.  90.  90.  90.]


In [6]:
cluster_for_traj = {}
chain_resid_traj = {}
contacts_traj = {}
norm_contacts_traj = {}
for tt in uni.trajectory:
    print(tt.frame,f'out of', len(uni.trajectory))

    chains_interactions = pd.DataFrame(columns=['chain_ai', 'chain_aj', 'min_distance'])
    
    chain_list = []
    for i in range(chains):
        chain_list.append(i)
    chain_comb = list(itertools.combinations(chain_list, 2))

    start_time = time.time()
    df_chain_ai, df_chain_aj, chain_distance = [], [], []
    for n in range(0, len(chain_comb)): # Circa 16 secondi per iterazione
        i = chain_comb[n][0]
        j = chain_comb[n][1]
        dist = distance_array(atom_Groups[i][0].positions,atom_Groups[j][0].positions,box).min()
        if dist <= Min_Distance:
            df_chain_ai.append(i)
            df_chain_aj.append(j)
            chain_distance.append(dist)

    chains_interactions['chain_ai'] = df_chain_ai
    chains_interactions['chain_aj'] = df_chain_aj
    chains_interactions['min_distance'] = chain_distance
    
    cluster_for_traj[tt.frame] = chains_interactions
    print("--- %s seconds ---" % (time.time() - start_time))

    trj_amino_interactions = []
    for i, j in zip(df_chain_ai, df_chain_aj):
        en_i, atoms_i, ai_resnumber = [], [], []
        for e, a in enumerate(AtomGroups[int(float(i))]):
            en_i.append(str(e) + '_' + str(i))
            ai_resnumber.append(e)
            atoms_i.append(a)
        
        en_j, atoms_j, aj_resnumber = [], [], []
        for e, a in enumerate(AtomGroups[int(float(j))]):
            en_j.append(str(e) + '_' + str(j))
            aj_resnumber.append(e)
            atoms_j.append(a)

        # Those two lists have the same shape, so iteration on atoms combination is the same about the resid
        # 11 aminoacids per chain -> 121 combinations
        en_comb = list(itertools.product(en_i, en_j)) # len 121
        resnumber_comb = list(itertools.product(ai_resnumber, aj_resnumber))
        atoms_comb = list(itertools.product(atoms_i, atoms_j))

        amino_distance = []
        for atoms in atoms_comb:            
            amino_distance.append(distance_array(atoms[0].positions,atoms[1].positions,box).min())

        # Build the consequent dataframe
        amino_interactions = pd.DataFrame(columns=['resid_chain_i', 'resid_chain_j', 'ai_resnumber', 'aj_resnumber', 'ai_resname', 'aj_resname', 'amino_distance'])

        amino_interactions['resid_chain_i'] = [x[0] for x in en_comb]
        amino_interactions['resid_chain_j'] = [x[1] for x in en_comb]
        amino_interactions['ai_resnumber'] = [x[0] for x in resnumber_comb]
        amino_interactions['aj_resnumber'] = [x[1] for x in resnumber_comb]
        amino_interactions['ai_resname'] = amino_interactions['ai_resnumber'].map(amino_dict_num)
        amino_interactions['aj_resname'] = amino_interactions['aj_resnumber'].map(amino_dict_num)
        amino_interactions['amino_distance'] = amino_distance
        
        trj_amino_interactions.append(amino_interactions)
    
        

    trj_amino_interactions = pd.concat(trj_amino_interactions, ignore_index=True)
    trj_amino_interactions = trj_amino_interactions.loc[trj_amino_interactions['amino_distance'] < Min_Distance]
    
    chain_resid_traj[tt.frame] = trj_amino_interactions

    contacts_histogram = {}
    frame_contacts_histogram = contact_extraction(trj_amino_interactions, contacts_histogram, pdb_resids)
    frame_contacts = make_contacts_histo_df(frame_contacts_histogram)

    contacts_norm = frame_contacts.div(frame_contacts.max().max())

    contacts_traj[tt.frame] = frame_contacts


    norm_contacts_traj[tt.frame] = contacts_norm
    print(contacts_norm)

print(norm_contacts_traj)



    #print(trj_amino_interactions)
print("--- %s seconds ---" % (time.time() - start_time))

0 out of 8
--- 35.64674210548401 seconds ---

Total amount of contacts made by TYR: 3

List of contacts based on aminoacid:
SER_10    1
THR_1     1
TYR_0     1

Total amount of contacts made by THR: 4

List of contacts based on aminoacid:
ILE_2    2
TYR_0    1
THR_1    1

Total amount of contacts made by ILE: 4

List of contacts based on aminoacid:
THR_1    2
ALA_3    1
ILE_2    1

Total amount of contacts made by ALA: 3

List of contacts based on aminoacid:
ALA_3    1
ALA_4    1
ILE_2    1

Total amount of contacts made by ALA: 1

List of contacts based on aminoacid:
ALA_3    1

Total amount of contacts made by LEU: 1

List of contacts based on aminoacid:
LEU_6    1

Total amount of contacts made by LEU: 1

List of contacts based on aminoacid:
LEU_5    1

Total amount of contacts made by SER: 0

List of contacts based on aminoacid:
Series([], )

Total amount of contacts made by PRO: 0

List of contacts based on aminoacid:
Series([], )

Total amount of contacts made by TYR: 1

List of 

In [11]:
pd.options.display.float_format = '{:.2f}'.format


for k in norm_contacts_traj:
    print(f'Timestep {k} \n\n {norm_contacts_traj[k]} \n\n\n\n')

Timestep 0 

         TYR  THR  ILE  ALA  ALA  LEU  LEU  SER  PRO  TYR  SER
TYR_0  0.50 0.50 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.50
THR_1  0.50 0.50 1.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00
ILE_2  0.00 1.00 0.50 0.50 0.00 0.00 0.00 0.00 0.00 0.00 0.00
ALA_3  0.00 0.00 0.50 0.50 0.50 0.00 0.00 0.00 0.00 0.00 0.00
ALA_4  0.00 0.00 0.00 0.50 0.00 0.00 0.00 0.00 0.00 0.00 0.00
LEU_5  0.00 0.00 0.00 0.00 0.00 0.00 0.50 0.00 0.00 0.00 0.00
LEU_6  0.00 0.00 0.00 0.00 0.00 0.50 0.00 0.00 0.00 0.00 0.00
TYR_9  0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.50 0.00
SER_10 0.50 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.00 




Timestep 1 

         TYR  THR  ILE  ALA  ALA  LEU  LEU  SER  PRO  TYR  SER
TYR_0  0.88 0.92 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.79 0.17
THR_1  0.92 0.83 0.92 0.00 0.00 0.00 0.00 0.00 0.00 0.42 0.00
ILE_2  0.00 0.92 0.67 0.88 0.00 0.00 0.00 0.00 0.00 1.00 0.00
ALA_3  0.00 0.00 0.88 0.79 0.71 0.00 0.00 0.00 0.00 0.00 0.00
ALA_4  0.00 0.00 0.00 0.71 0.58 0.62 

In [14]:
axn = len(norm_contacts_traj)


print(list(range(0, axn)))

[0, 1, 2, 3, 4, 5, 6, 7]


In [33]:
fig, ax = plt.subplots(nrows=axn, figsize=(12, axn*3.5))


for contacts in norm_contacts_traj.values():
    




#for contacts in norm_contacts_traj.values():
print(norm_contacts_traj[7])
contact_matrix = norm_contacts_traj[7].to_numpy()
fig, ax1 = plt.subplots(1,1, figsize = (10,10))
ax1.imshow(contact_matrix, interpolation='nearest', cmap = 'Blues')
plt.show()
print('coso')


        TYR  THR  ILE  ALA  ALA  LEU  LEU  SER  PRO  TYR  SER
TYR_0  0.62 0.51 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.50 1.00
THR_1  0.51 0.45 0.51 0.00 0.00 0.00 0.00 0.00 0.40 0.45 0.37
ILE_2  0.00 0.51 0.37 0.50 0.00 0.00 0.00 0.13 0.10 0.57 0.00
ALA_3  0.00 0.00 0.50 0.43 0.39 0.00 0.04 0.00 0.27 0.00 0.00
ALA_4  0.00 0.00 0.00 0.39 0.34 0.49 0.03 0.00 0.00 0.00 0.00
LEU_5  0.00 0.00 0.00 0.00 0.49 0.34 0.52 0.00 0.00 0.00 0.00
LEU_6  0.00 0.00 0.00 0.04 0.03 0.52 0.37 0.27 0.00 0.00 0.00
SER_7  0.00 0.00 0.13 0.00 0.00 0.00 0.27 0.35 0.33 0.00 0.00
PRO_8  0.00 0.40 0.10 0.27 0.00 0.00 0.00 0.33 0.19 0.40 0.00
TYR_9  0.50 0.45 0.57 0.00 0.00 0.00 0.00 0.00 0.40 0.23 0.34
SER_10 1.00 0.37 0.00 0.00 0.00 0.00 0.00 0.00 0.00 0.34 0.34
coso
